In [1]:
from allennlp.commands.elmo import ElmoEmbedder
import numpy as np
elmo = ElmoEmbedder(cuda_device = 0)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
import gc
import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import warnings

import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm, tqdm_notebook

In [4]:
elmo.cuda_device

0

In [27]:
%%time 

def cos_sim(vector_a, vector_b):

    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

tokens1 = ["I", "am", "a", "bad","boy"]
tokens2 = ["I", "am", "a", "good","girl"]
tokens3 = ["He", "is", "a", "good","man"]
tokens = [tokens1, tokens2, tokens3]
vectors1 = elmo.embed_sentence(tokens1)
vectors2 = elmo.embed_sentence(tokens2)
vectors3 = elmo.embed_sentence(tokens3)

print (cos_sim(vectors1[0][3], vectors2[0][3]) )
print (cos_sim(vectors2[0][3], vectors3[0][3]) )
print (cos_sim(vectors1[0][3], vectors3[0][3]) )

0.7512971974363012
0.9999999595786413
0.7512971974363012
CPU times: user 60.2 ms, sys: 47 µs, total: 60.2 ms
Wall time: 59 ms


In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x
tqdm.pandas()

train['Description'] = train['Description'].apply(clean_text)
test['Description'] = test['Description'].apply(clean_text)

In [42]:
%%time
sentences = train[train['Description'].notnull()]["Description"].progress_apply(lambda x: x.split()).values

batch_size  = 18
n_batches = len(sentences) // batch_size + 1
num =0
features = {}
wo_embed = 0
for b in tqdm(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = sentences[start:end]
    D = elmo.embed_batch(batch_pets)
    for i,pet_id in enumerate(batch_pets):
        num += 1
        try:
            features[num] = batch_preds[i][0][3] # last index control for the output from different layers 1 - 2 -3
        except:
            wo_embed +=1
            features[num] = np.zeros([1024])
    #if num > 20: break
print('train wo embed ', wo_embed)            

train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.columns = [f'elmo_{i}' for i in range(train_feats.shape[1])]


sentences = test[test['Description'].notnull()]["Description"].progress_apply(lambda x: x.split()).values
n_batches = len(sentences) // batch_size + 1
features = {}
num =0
wo_embed = 0
for b in tqdm(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = sentences[start:end]
    batch_preds = elmo.embed_batch(batch_pets)
    for i,pet_id in enumerate(batch_pets):
        num += 1
        try:
            features[num] = batch_preds[i][0][3]
        except:
            wo_embed +=1
            features[num] = np.zeros([1024])
    #if num > 20: break
            
print('test wo embed ', wo_embed)                 

test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.columns = [f'elmo_{i}' for i in range(test_feats.shape[1])]

100%|██████████| 1250/1250 [09:17<00:00,  2.24it/s]


train wo embed  1249


100%|█████████▉| 329/330 [02:07<00:00,  2.59it/s]


IndexError: list index out of range

In [36]:
train_feats.to_csv('elmo_train.csv')
test_feats.to_csv('elmo_test.csv')

In [40]:
start, end

(3948, 3960)